# Random Forest Model

In [2532]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from pandas import DataFrame


## Data Processing

In [2533]:
listStart = 0
listEnd = 0
predictData =  pd.DataFrame
playersList = pd.DataFrame
num_of_features = 10,
setSelectedFeatures = list[str]


In [2534]:
master_table_final = pd.read_csv("D:\School Programs\SE4482\master_table_final.csv")
master_table_final_fatigue = pd.read_csv("D:\School Programs\SE4482\master_table_final_fatigue.csv")

#Remove Duplicates:
master_table_final_fatigue.drop_duplicates(inplace=True)
master_table_final.drop_duplicates(inplace=True)

#Handle Missing Values:
#For simplicity, fill missing numeric values with the median and drop rows where categorical data (like Player or Tm) is missing.
numeric_cols = master_table_final_fatigue.select_dtypes(include=['float64', 'int64']).columns
master_table_final_fatigue[numeric_cols] = master_table_final_fatigue[numeric_cols].fillna(master_table_final_fatigue[numeric_cols].median())
master_table_final_fatigue.dropna(subset=['Player', 'Tm'], inplace=True)

#One-Hot Encoding: Convert categorical variables like Tm to a one-hot encoded format.
master_table_final_fatigue = pd.get_dummies(master_table_final_fatigue, columns=['Tm'])


remove_feature = ['W','Player', 'Share', 'WS', 'WS/48', 'Pts Won', 'Rank', 'First', 'usage_percentage', 'win_shares', 'defensive_win_shares' ]

def get_encode_data(X_data):

    X_encoded = pd.get_dummies(X_data, drop_first=True)
    
    # Ensure no missing values
    X_encoded.fillna(X_encoded.median(), inplace=True)

    return X_encoded

def process_data(data, setSelectedFeatures):

    
    data.drop_duplicates(inplace = True)
    numeric_cols = data.select_dtypes(include=['float64', 'int64']).columns
    data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())
    if 'Tm' in data.columns:
            data.dropna(subset=['Player', 'Tm'], inplace=True)
            data = pd.get_dummies(data, columns=['Tm'])
    else:
        print("'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.")
            
    X_data = data.drop(remove_feature, axis=1)  # Dropping non-feature columns and WS duplicate
    y_data = data['Share']  # MVP Share is the target

    X_encoded = get_encode_data(X_data)

    X_data = X_encoded[setSelectedFeatures]

    return y_data , X_data

## Feature Selection

In [2535]:
def retrieve_Feature_Mutual_Info(X_data, y_data):
    # Compute Mutual Information

    mi_scores = mutual_info_regression(X_data, y_data, random_state=42)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X_data.columns)
    mi_scores = mi_scores.sort_values(ascending=False)

    print("mi_scores")
    display(mi_scores)

    return mi_scores

In [2536]:
#Feature Selection

selected_features_fatigue = [
    # Basic stats
    'PTS', 'AST', 'TRB', 
    #Domain Knowledge stats
    'seed', 'value_over_replacement_player', "box_plus_minus","win_shares_per_48_minutes",
    # Consider MVP Fatigue
    'won1yearago', 'won2yearsago'
    ]
selected_features = [
    # Basic stats
    'PTS', 'AST', 'TRB',
    #Domain Knowledge stats
    'seed', 'value_over_replacement_player', "box_plus_minus","win_shares_per_48_minutes" ,
]

def setFeatures(max_number_of_features, fatigue, data):

    setDataSetX = data.drop(remove_feature, axis=1)  # Dropping non-feature columns and WS duplicate
    setDataSety = data['Share']

      #2. Handle Categorical Variables
    #If your dataset contains categorical features, you should encode them as Mutual Information can be computed for continuous variables. For simplicity, you might convert categorical variables to a one-hot encoded format.
    X_encoded = get_encode_data(setDataSetX)
    mi_scores = retrieve_Feature_Mutual_Info(X_encoded, setDataSety)

    
    if(fatigue):

      #Remove duplicates
      feature_list = selected_features_fatigue + mi_scores.index[:max_number_of_features].tolist()
      seen = set()
      unique_list = [x for x in feature_list if not (x in seen or seen.add(x))]
      
      return unique_list
      
    else:

      #Remove duplicates
      feature_list = selected_features + mi_scores.index[:max_number_of_features].tolist()
      seen = set()
      unique_list = [x for x in feature_list if not (x in seen or seen.add(x))]

      return feature_list


## Implementing Random Forest Model

In [2537]:
def useMVPYear(year, fatigue):
    
    if(fatigue):

        predictData = master_table_final_fatigue

    else: 
         predictData = master_table_final

    print("selected data:")
    display(predictData)

    setSelectedFeatures = setFeatures(10, fatigue, predictData)

    print("selected_features_fatigue")
    display(setSelectedFeatures )

    #map years to total data List
    start_index, end_index = year_indices[year]

    mvpYearData = predictData[start_index:end_index]

    y_proccesed_data, X_proccessed_data = process_data(mvpYearData,setSelectedFeatures)
    playersList = mvpYearData['Player']

    return playersList , y_proccesed_data , X_proccessed_data

year_indices = {
    1980: (0, 8),
    1981: (9, 40),
    1982: (41, 65),
    1983: (66, 88),
    1984: (89, 103),
    1985: (104, 126),
    1986: (127, 144),
    1987: (145, 161),
    1988: (162, 178),
    1989: (179, 197),
    1990: (198, 211),
    1991: (212, 231),
    1992: (232, 248),
    1993: (249, 262),
    1994: (263, 278),
    1995: (279, 292),
    1996: (293, 309),
    1997: (310, 329),
    1998: (330, 348),
    1999: (349, 369),
    2000: (370, 385),
    2001: (386, 402),
    2002: (403, 420),
    2003: (421, 433),
    2004: (434, 449),
    2005: (450, 464),
    2006: (465, 475),
    2007: (476, 492),
    2008: (493, 509),
    2009: (510, 520),
    2010: (521, 534),
    2011: (535, 547),
    2012: (548, 562),
    2013: (563, 578),
    2014: (579, 595),
    2015: (596, 607),
    2016: (608, 617),
    2017: (618, 628),
    2018: (629, 641),
    2019: (642, 653),
    2020: (654, 665),
    2021: (666, 678),
    2022: (679, 690),
    2023: (691, 703),
}

In [2538]:
def useAllMVPYears(fatigue):
    
    if(fatigue):

        predictData = master_table_final_fatigue

    else: 
         predictData = master_table_final

    # print("selected data:")
    # display(predictData)

    features_selected = setFeatures(10, fatigue, predictData)

    print("selected_features_fatigue")
    display(features_selected)

    y_proccesed_data, X_proccessed_data = process_data(predictData,features_selected)
    
    playersList = predictData['Player']

    return playersList , y_proccesed_data , X_proccessed_data

In [2539]:
#SET TEST CONDITIONS
fatigue = True
mvpYear = 0

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

#train model
players_proccessed_List , y_proccessed_data , X_proccessed_data = useAllMVPYears(fatigue)
X_train, X_test, y_train, y_test, players_list_train, players_list_test = train_test_split(X_proccessed_data, y_proccessed_data, players_proccessed_List, test_size=0.2, random_state=42)


# Fitting the model with the training data
rf_model.fit(X_train, y_train)

if mvpYear == 0:
    players_proccessed_List = players_list_test
    y_pred = rf_model.predict(X_test)

else:

    players_mvp_year_proccessed_List , y_mvp_year_data , X_mvp_year_data = useMVPYear(mvpYear, fatigue)
    players_proccessed_List = players_mvp_year_proccessed_List

    y_pred = rf_model.predict(X_mvp_year_data)

# print("data:")
# display(X_proccessed_data)
print("shares:")
display(y_proccessed_data)
print("players:")
display(players_proccessed_List)



# Predicting the MVP Share for the test set
predicted_mvp_shares = pd.DataFrame({
    'Player': players_proccessed_List,
    'Predicted MVP Share': y_pred.flatten()
})

top_mvp_candidates = predicted_mvp_shares.sort_values(by='Predicted MVP Share', ascending=False).head(len(predicted_mvp_shares))
print('Predicted for year', mvpYear)
print(top_mvp_candidates)



mi_scores


player_efficiency_rating         0.285007
win_shares_per_48_minutes        0.266906
value_over_replacement_player    0.250951
box_plus_minus                   0.242868
offensive_box_plus_minus         0.199635
                                   ...   
team_Boston Celtics              0.000000
team_Brooklyn Nets               0.000000
team_Charlotte Bobcats           0.000000
team_Cleveland Cavaliers         0.000000
team_Washington Wizards          0.000000
Name: MI Scores, Length: 107, dtype: float64

selected_features_fatigue


['PTS',
 'AST',
 'TRB',
 'seed',
 'value_over_replacement_player',
 'box_plus_minus',
 'win_shares_per_48_minutes',
 'won1yearago',
 'won2yearsago',
 'player_efficiency_rating',
 'offensive_box_plus_minus',
 'offensive_win_shares',
 'W/L%',
 'true_shooting_percentage']

'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
shares:


0      0.665
1      0.143
2      0.086
3      0.068
4      0.009
       ...  
699    0.005
700    0.003
701    0.002
702    0.001
703    0.001
Name: Share, Length: 704, dtype: float64

players:


296    Hakeem Olajuwon
81        Bernard King
77         Jeff Ruland
208         Larry Bird
318      Anthony Mason
            ...       
108     Terry Cummings
256        Karl Malone
56        Ray Williams
494         Chris Paul
519         Tim Duncan
Name: Player, Length: 141, dtype: object

Predicted for year 0
               Player  Predicted MVP Share
212    Michael Jordan              0.91905
521      LeBron James              0.91780
145     Magic Johnson              0.81364
310       Karl Malone              0.80935
514        Chris Paul              0.70309
..                ...                  ...
328       Steve Smith              0.00517
346         Rik Smits              0.00493
54       Bernard King              0.00474
277  Latrell Sprewell              0.00454
369      Vince Carter              0.00380

[141 rows x 2 columns]


## Cross Validation

In [2540]:
scores = cross_val_score(rf_model, X_train, y_train, cv=10, scoring='neg_mean_squared_error')

## Evaluate the Model



In [2541]:
# Calculate the model performance metrics
def eval_model( y_compare ,y_pred):


    mse = mean_squared_error(y_compare, y_pred)

    r2 = r2_score(y_compare, y_pred)

    mae =  mean_absolute_error(y_compare, y_pred)

    print(f"Num of Features: {num_of_features}")
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")
    print(f"Mean Absolute Error: {mae}")

    mse_scores = -scores  # Make MSE scores positive
    print("MSE scores from each fold:", mse_scores)
    print("Average MSE:", mse_scores.mean())
    print("Standard deviation of MSE:", mse_scores.std())


if mvpYear == 0:

    eval_model(y_test, y_pred)


else:

    eval_model(y_mvp_year_data, y_pred)


Num of Features: (10,)
Mean Squared Error: 0.024610976741843975
R-squared: 0.6287704341279216
Mean Absolute Error: 0.09695205673758867
MSE scores from each fold: [0.0304569  0.03144717 0.01771811 0.02799414 0.01933689 0.02697519
 0.04014111 0.01885441 0.02350702 0.01601856]
Average MSE: 0.025244949368615283
Standard deviation of MSE: 0.007201551888279556


## Feature Importance

In [2542]:
# Get feature importance
feature_importances = rf_model.feature_importances_
features = pd.DataFrame({'Feature': X_train.columns, 'Importance': feature_importances})
features.sort_values(by='Importance', ascending=False, inplace=True)

print(features)


                          Feature  Importance
4   value_over_replacement_player    0.369649
9        player_efficiency_rating    0.113489
6       win_shares_per_48_minutes    0.088732
12                           W/L%    0.075116
3                            seed    0.074911
0                             PTS    0.053699
5                  box_plus_minus    0.049127
2                             TRB    0.042988
1                             AST    0.037411
13       true_shooting_percentage    0.035811
11           offensive_win_shares    0.027079
10       offensive_box_plus_minus    0.016380
7                     won1yearago    0.008138
8                    won2yearsago    0.007469


In [2543]:
import pandas as pd


def extract_actual_mvps(year_indices, dataset):

    actual_mvps = {}
    for year, (start_index, _) in year_indices.items():
        # Assuming the MVP is the first player listed in the segment for each year
        actual_mvp = dataset.iloc[start_index]['Player']
        actual_mvps[year] = actual_mvp
    return actual_mvps


actualVPS = extract_actual_mvps(year_indices, master_table_final_fatigue)
print(actualVPS)

def count_correct_mvp_predictions(year_indices, fatigue, rf_model, actual_mvps_dict):
    correct_predictions = 0
    
    for year in year_indices.keys():
        # Use the model to predict MVP shares for the given year
        players_mvp_year_processed_list, _, X_mvp_year_data = useMVPYear(year, fatigue)
        y_pred = rf_model.predict(X_mvp_year_data)
        
        # Create a DataFrame of predicted shares
        predicted_mvp_shares = pd.DataFrame({
            'Player': players_mvp_year_processed_list,
            'Predicted MVP Share': y_pred.flatten()
        }).sort_values(by='Predicted MVP Share', ascending=False)
        
        # Assume the player with the highest predicted share is the predicted MVP
        predicted_mvp = predicted_mvp_shares.iloc[0]['Player']
        
        # Get the actual MVP for the year from the actual_mvps_dict dictionary
        actual_mvp = actual_mvps_dict[year]
        
        # Check if the prediction was correct
        if predicted_mvp == actual_mvp:
            correct_predictions += 1
    
    return correct_predictions


correct_predictions = count_correct_mvp_predictions(year_indices,fatigue,rf_model,actualVPS)

print("here",correct_predictions)
print(correct_predictions/len(actualVPS))

{1980: 'Kareem Abdul-Jabbar', 1981: 'Julius Erving', 1982: 'Moses Malone', 1983: 'Moses Malone', 1984: 'Larry Bird', 1985: 'Larry Bird', 1986: 'Larry Bird', 1987: 'Magic Johnson', 1988: 'Michael Jordan', 1989: 'Magic Johnson', 1990: 'Magic Johnson', 1991: 'Michael Jordan', 1992: 'Michael Jordan', 1993: 'Charles Barkley', 1994: 'Hakeem Olajuwon', 1995: 'David Robinson', 1996: 'Michael Jordan', 1997: 'Karl Malone', 1998: 'Michael Jordan', 1999: 'Karl Malone', 2000: "Shaquille O'Neal", 2001: 'Allen Iverson', 2002: 'Tim Duncan', 2003: 'Tim Duncan', 2004: 'Kevin Garnett', 2005: 'Steve Nash', 2006: 'Steve Nash', 2007: 'Dirk Nowitzki', 2008: 'Kobe Bryant', 2009: 'LeBron James', 2010: 'LeBron James', 2011: 'Derrick Rose', 2012: 'LeBron James', 2013: 'LeBron James', 2014: 'Kevin Durant', 2015: 'Stephen Curry', 2016: 'Stephen Curry', 2017: 'Russell Westbrook', 2018: 'James Harden', 2019: 'Giannis Antetokounmpo', 2020: 'Giannis Antetokounmpo', 2021: 'Nikola Jokić', 2022: 'Nikola Jokić', 2023: 'Jo

,Rank,Player,Age,First,Pts Won,Pts Max,Share,G,MP,PTS,...,Tm_PHO,Tm_POR,Tm_SAC,Tm_SAS,Tm_SDC,Tm_SEA,Tm_TOR,Tm_UTA,Tm_WAS,Tm_WSB
0,1,Kareem Abdul-Jabbar,32,147.0,147.0,221,0.665,82,38.3,24.8,...,False,False,False,False,False,False,False,False,False,False
1,2,Julius Erving,29,31.5,31.5,221,0.143,78,36.1,26.9,...,False,False,False,False,False,False,False,False,False,False
2,3,George Gervin,27,19.0,19.0,221,0.086,78,37.6,33.1,...,False,False,False,True,False,False,False,False,False,False
3,4,Larry Bird,23,15.0,15.0,221,0.068,82,36.0,21.3,...,False,False,False,False,False,False,False,False,False,False
4,5,Dan Roundfield,26,2.0,2.0,221,0.009,81,32.0,16.5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,9,Stephen Curry,34,0.0,5.0,1000,0.005,56,34.7,29.4,...,False,False,False,False,False,False,False,False,False,False
700,10,Jimmy Butler,33,0.0,3.0,1000,0.003,64,33.4,22.9,...,False,False,False,False,False,False,False,False,False,False
701,11,De'Aaron Fox,25,0.0,2.0,1000,0.002,73,33.4,25.0,...,False,False,True,False,False,False,False,False,False,False
702,12,Ja Morant,23,0.0,1.0,1000,0.001,61,31.9,26.2,...,False,False,False,False,False,False,False,False,False,False


mi_scores


player_efficiency_rating         0.285007
win_shares_per_48_minutes        0.266906
value_over_replacement_player    0.250951
box_plus_minus                   0.242868
offensive_box_plus_minus         0.199635
                                   ...   
team_Boston Celtics              0.000000
team_Brooklyn Nets               0.000000
team_Charlotte Bobcats           0.000000
team_Cleveland Cavaliers         0.000000
team_Washington Wizards          0.000000
Name: MI Scores, Length: 107, dtype: float64

selected_features_fatigue


...

player_efficiency_rating         0.285007
win_shares_per_48_minutes        0.266906
value_over_replacement_player    0.250951
box_plus_minus                   0.242868
offensive_box_plus_minus         0.199635
                                   ...   
team_Boston Celtics              0.000000
team_Brooklyn Nets               0.000000
team_Charlotte Bobcats           0.000000
team_Cleveland Cavaliers         0.000000
team_Washington Wizards          0.000000
Name: MI Scores, Length: 107, dtype: float64

selected_features_fatigue


['PTS',
 'AST',
 'TRB',
 'seed',
 'value_over_replacement_player',
 'box_plus_minus',
 'win_shares_per_48_minutes',
 'won1yearago',
 'won2yearsago',
 'player_efficiency_rating',
 'offensive_box_plus_minus',
 'offensive_win_shares',
 'W/L%',
 'true_shooting_percentage']

'Tm' column not found. Skipping dropna and get_dummies for 'Tm' column.
here 35
0.7954545454545454


C:\Users\warre\AppData\Local\Temp\ipykernel_12028\1527606789.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop_duplicates(inplace = True)
C:\Users\warre\AppData\Local\Temp\ipykernel_12028\1527606789.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numeric_cols] = data[numeric_cols].fillna(data[numeric_cols].median())
